In [ ]:
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
MOVIE_PATH = DATA_PATH +'Control 050721/'
GRAPH_PATH = 'C:Users/amityu/Gel_Sheet_Graph/'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from  skimage.filters import gaussian
from tqdm.notebook import trange, tqdm
import pandas as pd
import movie_structure
from  concurrent.futures import ThreadPoolExecutor
import preprocessing
import autocorrelation_and_structure_factor_IK_20220103 as iac
import importlib
import seaborn as sns
from skimage import filters
#from p_tqdm import p_map
from scipy.ndimage import median_filter


In [ ]:
importlib.reload(plotly)

In [ ]:
gel = np.load(MOVIE_PATH +'np/gel_norm.npy', mmap_mode='r+')
mask = np.load(MOVIE_PATH +'np/mask.npy', mmap_mode='r+')

In [ ]:
flat = gel.reshape(-1)
plt.hist(flat)
plt.show()

In [ ]:
print ('gel min %f gel max %f' %(np.min(gel),np.max(gel)))


In [ ]:

t1 = 63
x = gel[t1].reshape(-1)#[1:-1:20]
t2 = 64
y = gel[t2].reshape(-1)#[1:-1:20]
t3 = 65
z = gel[t1].reshape(-1)#[1:-1:20]
t4 = 66
w = gel[t1].reshape(-1)#[1:-1:20]

hist_df = pd.concat(axis=0, ignore_index=True, objs=[
    pd.DataFrame.from_dict({'value': x, 'Time': str(t1)}),
    pd.DataFrame.from_dict({'value': y, 'Time': str(t2)}),
    pd.DataFrame.from_dict({'value': z, 'Time': str(t3)}),
    pd.DataFrame.from_dict({'value': w, 'Time': str(t4)})
])
fig, ax = plt.subplots()
sns.histplot(
    data=hist_df, x='value', hue='Time', multiple='dodge',
     ax=ax, stat='density', bins=range(0,600,30), element='step'
)
ax.set_xlim([0, 600])
ax.set_title('Time points %d, %d, %d %d Histogram '%(t1,t2,t3,t4))


In [ ]:
t = 65
flat = gel[t].reshape(-1)[1:-1:10]
n, bins, patches = plt.hist(flat,cumulative=False, density=True)
plt.title('Time poind %d Histogram '%t)
plt.show()

In [ ]:
min_intensity = bins[1]
max_intensity = 10000

threshold_gel = gaussian(gel[65],3)
threshold_gel[threshold_gel<min_intensity] =0
threshold_gel[threshold_gel>max_intensity] = 0
threshold_gel[np.bitwise_and(threshold_gel>= min_intensity , threshold_gel<= max_intensity)] =1

# np.save(DATA_PATH + 'tmp/t_gel.npy', threshold_gel)



In [ ]:
#import napari
viewer = napari.Viewer()
image = gel[:,:,:,256]
viewer.add_image(image, name='65')
napari.utils.nbscreenshot(viewer)

In [ ]:
for t in range(5,85,10):
    viewer.add_image(gel[t,:,:,256], name = 'gel %d'%t )
    viewer.add_image(mask[t,:,:,256], name = 'mask %d'%t )



In [ ]:
import matplotlib.image
for t in range(5,86,10):


    matplotlib.image.imsave(MOVIE_PATH + 'tmp/gel %t x =256.jpeg', gel[t,:,:,256])
    matplotlib.image.imsave(MOVIE_PATH + 'tmp/gel %t x =256.jpeg', gel[t,:,:,256])#%%
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
MOVIE_PATH = DATA_PATH +'Control 050721/'
GRAPH_PATH = 'C:Users/amityu/Gel_Sheet_Graph/'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from  skimage.filters import gaussian
from tqdm.notebook import trange, tqdm
import pandas as pd
import movie_structure
from  concurrent.futures import ThreadPoolExecutor
import preprocessing
import autocorrelation_and_structure_factor_IK_20220103 as iac
import importlib
import seaborn as sns
from skimage import filters
#from p_tqdm import p_map


In [ ]:
gel = np.load(MOVIE_PATH +'np/gel_norm.npy', mmap_mode='r+')
mask = np.load(MOVIE_PATH +'np/mask.npy', mmap_mode='r+')

In [ ]:
flat = gel.reshape(-1)
plt.hist(flat)
plt.show()

In [ ]:
print ('gel min %f gel max %f' %(np.min(gel),np.max(gel)))


In [ ]:

t1 = 63
x = gel[t1].reshape(-1)#[1:-1:20]
t2 = 64
y = gel[t2].reshape(-1)#[1:-1:20]
t3 = 65
z = gel[t1].reshape(-1)#[1:-1:20]
t4 = 66
w = gel[t1].reshape(-1)#[1:-1:20]

hist_df = pd.concat(axis=0, ignore_index=True, objs=[
    pd.DataFrame.from_dict({'value': x, 'Time': str(t1)}),
    pd.DataFrame.from_dict({'value': y, 'Time': str(t2)}),
    pd.DataFrame.from_dict({'value': z, 'Time': str(t3)}),
    pd.DataFrame.from_dict({'value': w, 'Time': str(t4)})
])
fig, ax = plt.subplots()
sns.histplot(
    data=hist_df, x='value', hue='Time', multiple='dodge',
     ax=ax, stat='density', bins=range(0,600,30), element='step'
)
ax.set_xlim([0, 600])
ax.set_title('Time points %d, %d, %d Histogram '%(t1,t2, t3))


In [ ]:
t = 65
flat = gel[t].reshape(-1)[1:-1:10]
n, bins, patches = plt.hist(flat,cumulative=False, density=True)
plt.title('Time poind %d Histogram '%t)
plt.show()

In [ ]:
min_intensity = bins[1]
max_intensity = 10000

threshold_gel = gaussian(gel[65],3)
threshold_gel[threshold_gel<min_intensity] =0
threshold_gel[threshold_gel>max_intensity] = 0
threshold_gel[np.bitwise_and(threshold_gel>= min_intensity , threshold_gel<= max_intensity)] =1

# np.save(DATA_PATH + 'tmp/t_gel.npy', threshold_gel)



In [ ]:
#import napari
viewer = napari.Viewer()
image = gel[65,:,:,256]
viewer.add_image(image, name='65')
napari.utils.nbscreenshot(viewer)

In [ ]:
for t in range(5,85,10):
    viewer.add_image(gel[t,:,:,256], name = 'gel %d'%t )
    viewer.add_image(mask[t,:,:,256], name = 'mask %d'%t )



In [ ]:
import matplotlib.image

fig, ax = plt.subplots(10,2, figsize = (30,200))
plt.subplots_adjust(hspace=1)
t= 60
x = 30
for col in range(ax.shape[0]):
    sns.heatmap(gel[t,:,:,x], ax=ax[col, 0], xticklabels= [], yticklabels=50, cbar=False)
    sns.heatmap(median_filter(mask[t,:,:,x],5), ax=ax[col, 1], xticklabels= [], yticklabels=50, cbar=False)
    ax[col, 0].set_title('Gel T= %d'%t, fontsize = 12)
    ax[col, 1].set_title('Mask T = %d'%t, fontsize = 12)
    t+=1


In [ ]:
from  concurrent.futures import ThreadPoolExecutor
gradient_table = np.zeros((len(gel), gel.shape[2]))
height_table = np.zeros((len(gel), gel.shape[2]))
height_deviation = np.zeros((len(gel), gel.shape[2]))

def x(t):
    print('proces %d starts'%t)
    #flat = gel[t].reshape(-1)[1:-1:10]
    # n, bins, patches = plt.hist(flat,cumulative=True,  density=True)
    #hist, bins = np.histogram(flat, density=True)
    #min_intensity = bins[2]

    max_intensity = 10000

    gel_time_point = gel[t]
    for i in range(gel_time_point.shape[1]):
        for j in range(gel_time_point.shape[2]):
            z_line = gel_time_point[:,i,j]
            min_intensity = filters.threshold_li(z_line)

            z_line[z_line < min_intensity] =0
            z_line[z_line>max_intensity] = 0
            z_line[np.bitwise_and(z_line>= min_intensity , z_line<= max_intensity)] =1
    plane = movie_structure.Vplane(gel_time_point[:,:,256], gel_time_point[:,:,256])
    height = plane.set_height()

    height_table[t] = height
    height_deviation[t] = height - np.nanmean(height)

    gradient_table[t] = np.gradient(height)
    print('proces %d ends'%t)
    return height

In [ ]:

with ThreadPoolExecutor(max_workers=8) as e:
    for t in trange(len(gel)):
        e.submit(x,t)



In [ ]:
for t in trange(len(gel)):
    x(t)

np.save(MOVIE_PATH +'tmp/gradient_table.npy',gradient_table)
np.save(MOVIE_PATH +'tmp/height_table.npy', height_table)
np.save(MOVIE_PATH +'tmp/height_deviation.npy', height_deviation )


In [ ]:
np.save (MOVIE_PATH +'tmp/mask.npy',gel)


In [ ]:
gradient_table = np.zeros((len(gel), gel.shape[2]))
height_table = np.zeros((len(gel), gel.shape[2]))
height_deviation = np.zeros((len(gel), gel.shape[2]))

In [ ]:
'''x_index = 320

for tp in tqdm(gel):
    plane = movie_structure.Vplane(tp[:,:,x_index], tp[:,:,x_index])
    height = plane.set_height()

    height_table[t] = height
    height_deviation[t] = height - np.nanmean(height)

    gradient_table[t] = np.gradient(height)
'''

In [ ]:
np.save(MOVIE_PATH + 'tmp/mask.npy', gel)

In [ ]:
#gradient_table = gradient_table[range(len(gradient_table)-1,0,-1)]
plt.figure(figsize=(30,30))
plt.imshow(gradient_table, vmin= -1, vmax =1, aspect = 'auto')
plt.xlabel('x - axe',  fontsize = 40)
plt.ylabel('Time', fontsize = 40)
plt.title('Boundary Gradient', fontsize = 40)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(height_deviation, vmin = -5, vmax =5 ,aspect = 'auto')
plt.xlabel('x - axe',fontsize = 40)
plt.ylabel('Time', fontsize = 40)
plt.title('Deviation from mean height',fontsize = 40)
plt.colorbar()
plt.show()



In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(height_table, aspect = 'auto')
plt.xlabel('x - axe',fontsize = 40)
plt.ylabel('Time', fontsize = 40)
plt.title('Gel Height',fontsize = 40)
plt.colorbar()
plt.show()


In [ ]:
mean_height = np.zeros(len(height_table))
for t in trange(len(height_table)):
    mean_height[t] = np.nanmean(height_table[t])
plt.scatter(range(len(mean_height)),mean_height)
plt.xlabel('Time')
plt.ylabel('Height')
plt.show()

In [ ]:
x1, y1 = (0,40)
x2,y2 = (90,140)

a = (y2-y1)/(x2-x1)
y1 = a

In [ ]:
auto = iac.spatial_autocorr(gaussian(gel[:,20,:,256],3), range(160),range(512))
norm_auto = ((auto[0] - auto[0].min())/(auto[0].max() -auto[0].min())-0.5)*2
plt.imshow(norm_auto ,  cmap= cm.get_cmap('RdBu'),aspect = 'auto')
plt.ylabel('Time', fontsize = 40)
plt.colorbar()
plt.show()

In [ ]:
plt.hist(auto[0])
plt.show()

In [ ]:
mean_list = []
for t in range(len(gel)):
    mean_list.append((gel[t].min()))
plt.plot(mean_list)
plt.show()

In [ ]:
import libpysal

import numpy as np

#np.random.seed(10)
image = gel[20,20,:,:].astype('uint16')
def autocorr(image):
    w = libpysal.weights.lat2W(*image.shape)

    y = image.reshape(-1,1)

    y = np.copy(y)
    from esda.moran import Moran_Local

    lm = Moran_Local(y, w, transformation = "r", permutations = 99)
    return lm.Is.reshape(*image.shape)


In [ ]:
image = gel[20,20,:,:].astype('uint16')
output_image = autocorr(image)

plt.figure(figsize=(30,30))
plt.imshow(output_image, vmin =-5, vmax=5, aspect='auto',origin='lower')
plt.ylabel('Time',fontsize = 40)
plt.colorbar()
plt.show()


In [ ]:
output_image = lm.Is.reshape(*image.shape)
plt.imshow(output_image)
plt.colorbar()
plt.show()


In [ ]:
import matplotlib.image as mpimg
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as pl

image = output_image.copy()
def power_spectrum(image):
    npix = image.shape[0]

    fourier_image = np.fft.fftn(image)
    fourier_amplitudes = np.abs(fourier_image)**2

    kfreq = np.fft.fftfreq(npix) * npix
    kfreq2D = np.meshgrid(kfreq, kfreq)
    knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

    knrm = knrm.flatten()
    fourier_amplitudes = fourier_amplitudes.flatten()

    kbins = np.arange(0.5, npix//2+1, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                         statistic = "mean",
                                         bins = kbins)
    Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)
    return (kvals,Abins)


In [ ]:
image = output_image.copy()
kvals, Abins =  power_spectrum(image)

pl.loglog(kvals, Abins)
pl.xlabel("$k$")
pl.ylabel("$P(k)$")
pl.tight_layout()
plt.show()

In [ ]:
kvals_ls = []
abins_ls = []
for t in trange(5,41,5):
    image = gel[t,20,:,:].astype('uint16')
    output_image = autocorr(image)
    image = output_image.copy()
    kvals, Abins =  power_spectrum(image)
    kvals_ls.append(kvals)
    abins_ls.append(Abins)

    pl.loglog(kvals, Abins, label=str(t))
    pl.xlabel("$k$")
    pl.ylabel("$P(k)$")
    pl.tight_layout()
    pl.loglog(kvals, Abins, ls="None")

df = pd.DataFrame(columns=['kval','abins'])
df['kval'] = kvals_ls
df['abins'] = abins_ls
df.to_csv(GRAPH_PATH + 'spectrum/spectrum.csv')
plt.title('Power Spectrum')
plt.legend()
plt.show()

In [ ]:
#plt.imshow(image, cmap='Set1')
plt.plot(np.arange(10), np.arange(10)/10, color = cm.get_cmap('Set1'))
plt.show()

In [ ]:
zy = gel[5][:,:,256]
plt.imshow(zy)
plt.colorbar()
plt.show()


In [ ]:
plt.hist(zy[:,40])

In [ ]:
plt.plot(zy[:,10])
plt.show()

In [ ]:
from skimage import filters
filters.threshold_li(zy[25:,10])

In [ ]:
zy = gel[20][:,:,444]
plt.imshow(zy)
plt.colorbar()
plt.show()


In [ ]:
plt.hist(zy[:,40])


In [ ]:
plt.plot(zy[:,40])
plt.show()

In [ ]:
h = np.zeros((len(gel), gel.shape[2], gel.shape[3]))


def p(time):
    tp = movie_structure.TimePoint(gel[time])
    h[time] = tp.set_height()

with ThreadPoolExecutor() as e:
    for t in trange(len(gel)):
        e.submit(p,t)



In [ ]:
np.save(MOVIE_PATH + 'tmp/h.npy',h)

In [ ]:
mean_height = np.zeros(len(h))
for t in trange(len(h)):
    mean_height[t] = np.nanmean(h[t])

In [ ]:
plt.scatter(range(len(h)),mean_height)
plt.xlabel('Time')
plt.ylabel('Height')
plt.show()

In [ ]:
g = np.load(MOVIE_PATH +'tmp/gel_uint32.npy')

In [ ]:
g.max()

In [ ]:
viewer.add_image(g[0,:,:,20], name='uint32')

In [ ]:
importlib.reload(preprocessing)

In [ ]:
gel = preprocessing.normalize_to_background(gel,0,20,(0,0),(10,10))

In [ ]:
np.save(MOVIE_PATH +'tmp/gel_norm.npy' , gel)

In [ ]:
gel[:,0:20,:,:] = gel[:,-20:,:,:].copy()

In [ ]:
importlib.reload(movie_structure)


In [ ]:
df = pd.read_csv(MOVIE_PATH + 'height.csv')

data = df.drop(['order','Unnamed: 0'],axis=1)
ax = sns.heatmap(data,xticklabels =50, yticklabels=20, cmap ='plasma')
plt.xlabel('X')
plt.ylabel('Time')
plt.title('Gel Height')
ax.invert_yaxis()


In [ ]:
import seaborn as sns
df = pd.read_csv(MOVIE_PATH + 'square.csv')

data = df.drop(['order','Unnamed: 0'],axis=1)
data = data.applymap(lambda x: x/512)
ax = sns.heatmap(data,xticklabels =50, yticklabels=20, cmap ='plasma')
plt.xlabel('X')
plt.ylabel('Time')
plt.title('Gel Square deviation from mean height')
ax.invert_yaxis()



In [ ]:
df = pd.read_csv(MOVIE_PATH + 'curvature.csv')

data = df.drop(['order','Unnamed: 0'],axis=1)
ax = sns.heatmap(data,xticklabels =50, yticklabels=20, cmap ='plasma')
plt.xlabel('X')
plt.ylabel('Time')
plt.title('Abs curvature of boundary')
ax.invert_yaxis()


In [ ]:
data = df.drop(['order'],axis=1)
sns.heatmap(data)

In [ ]:
importlib.reload(movie_structure)


In [ ]:
fig, axes = plt.subplots(2,5, figsize=(30,12))

t=10
for row in axes:
    for col in row:
        if t >90: continue
        tp = movie_structure.TimePoint(gel[t], mask[t])
        surface = tp.set_height_surface()
        sns.heatmap(surface - np.mean(surface),ax= col, cmap= 'seismic', vmin = -200, vmax =200, xticklabels=50, yticklabels=50, square=True)
        col.set_title('Surface height deviation Time point %d'%t, fontsize = 16)
        t+=10

plt.show()


$curvature = \frac{\frac{\partial^2 z}{\partial x^2} \frac{\partial^2 z}{\partial y^2} - \left(\frac{\partial^2 z}{\partial x \partial y}\right)^2}{\left(1 + \left(\frac{\partial z}{\partial x}\right)^2 + \left(\frac{\partial z}{\partial y}\right)^2\right)^{3/2}}$

In [ ]:
from  concurrent.futures import ThreadPoolExecutor
gradient_table = np.zeros((len(gel), gel.shape[2]))
height_table = np.zeros((len(gel), gel.shape[2]))
height_deviation = np.zeros((len(gel), gel.shape[2]))

def x(t):
    print('proces %d starts'%t)
    #flat = gel[t].reshape(-1)[1:-1:10]
    # n, bins, patches = plt.hist(flat,cumulative=True,  density=True)
    #hist, bins = np.histogram(flat, density=True)
    #min_intensity = bins[2]

    max_intensity = 10000

    gel_time_point = gel[t]
    for i in range(gel_time_point.shape[1]):
        for j in range(gel_time_point.shape[2]):
            z_line = gel_time_point[:,i,j]
            min_intensity = filters.threshold_li(z_line)

            z_line[z_line < min_intensity] =0
            z_line[z_line>max_intensity] = 0
            z_line[np.bitwise_and(z_line>= min_intensity , z_line<= max_intensity)] =1
    plane = movie_structure.Vplane(gel_time_point[:,:,256], gel_time_point[:,:,256])
    height = plane.set_height()

    height_table[t] = height
    height_deviation[t] = height - np.nanmean(height)

    gradient_table[t] = np.gradient(height)
    print('proces %d ends'%t)
    return height

In [ ]:

with ThreadPoolExecutor(max_workers=8) as e:
    for t in trange(len(gel)):
        e.submit(x,t)



In [ ]:
for t in trange(len(gel)):
    x(t)

np.save(MOVIE_PATH +'tmp/gradient_table.npy',gradient_table)
np.save(MOVIE_PATH +'tmp/height_table.npy', height_table)
np.save(MOVIE_PATH +'tmp/height_deviation.npy', height_deviation )


In [ ]:
np.save (MOVIE_PATH +'tmp/mask.npy',gel)


In [ ]:
gradient_table = np.zeros((len(gel), gel.shape[2]))
height_table = np.zeros((len(gel), gel.shape[2]))
height_deviation = np.zeros((len(gel), gel.shape[2]))

In [ ]:
'''x_index = 320

for tp in tqdm(gel):
    plane = movie_structure.Vplane(tp[:,:,x_index], tp[:,:,x_index])
    height = plane.set_height()

    height_table[t] = height
    height_deviation[t] = height - np.nanmean(height)

    gradient_table[t] = np.gradient(height)
'''

In [ ]:
np.save(MOVIE_PATH + 'tmp/mask.npy', gel)

In [ ]:
#gradient_table = gradient_table[range(len(gradient_table)-1,0,-1)]
plt.figure(figsize=(30,30))
plt.imshow(gradient_table, vmin= -1, vmax =1, aspect = 'auto')
plt.xlabel('x - axe',  fontsize = 40)
plt.ylabel('Time', fontsize = 40)
plt.title('Boundary Gradient', fontsize = 40)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(height_deviation, vmin = -5, vmax =5 ,aspect = 'auto')
plt.xlabel('x - axe',fontsize = 40)
plt.ylabel('Time', fontsize = 40)
plt.title('Deviation from mean height',fontsize = 40)
plt.colorbar()
plt.show()



In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(height_table, aspect = 'auto')
plt.xlabel('x - axe',fontsize = 40)
plt.ylabel('Time', fontsize = 40)
plt.title('Gel Height',fontsize = 40)
plt.colorbar()
plt.show()


In [ ]:
mean_height = np.zeros(len(height_table))
for t in trange(len(height_table)):
    mean_height[t] = np.nanmean(height_table[t])
plt.scatter(range(len(mean_height)),mean_height)
plt.xlabel('Time')
plt.ylabel('Height')
plt.show()

In [ ]:
x1, y1 = (0,40)
x2,y2 = (90,140)

a = (y2-y1)/(x2-x1)
y1 = a

In [ ]:
auto = iac.spatial_autocorr(gaussian(gel[:,20,:,256],3), range(160),range(512))
norm_auto = ((auto[0] - auto[0].min())/(auto[0].max() -auto[0].min())-0.5)*2
plt.imshow(norm_auto ,  cmap= cm.get_cmap('RdBu'),aspect = 'auto')
plt.ylabel('Time', fontsize = 40)
plt.colorbar()
plt.show()

In [ ]:
plt.hist(auto[0])
plt.show()

In [ ]:
mean_list = []
for t in range(len(gel)):
    mean_list.append((gel[t].min()))
plt.plot(mean_list)
plt.show()

In [ ]:
import libpysal

import numpy as np

#np.random.seed(10)
image = gel[20,20,:,:].astype('uint16')
def autocorr(image):
    w = libpysal.weights.lat2W(*image.shape)

    y = image.reshape(-1,1)

    y = np.copy(y)
    from esda.moran import Moran_Local

    lm = Moran_Local(y, w, transformation = "r", permutations = 99)
    return lm.Is.reshape(*image.shape)


In [ ]:
image = gel[20,20,:,:].astype('uint16')
output_image = autocorr(image)

plt.figure(figsize=(30,30))
plt.imshow(output_image, vmin =-5, vmax=5, aspect='auto',origin='lower')
plt.ylabel('Time',fontsize = 40)
plt.colorbar()
plt.show()


In [ ]:
output_image = lm.Is.reshape(*image.shape)
plt.imshow(output_image)
plt.colorbar()
plt.show()


In [ ]:
import matplotlib.image as mpimg
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as pl

image = output_image.copy()
def power_spectrum(image):
    npix = image.shape[0]

    fourier_image = np.fft.fftn(image)
    fourier_amplitudes = np.abs(fourier_image)**2

    kfreq = np.fft.fftfreq(npix) * npix
    kfreq2D = np.meshgrid(kfreq, kfreq)
    knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

    knrm = knrm.flatten()
    fourier_amplitudes = fourier_amplitudes.flatten()

    kbins = np.arange(0.5, npix//2+1, 1.)
    kvals = 0.5 * (kbins[1:] + kbins[:-1])
    Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                         statistic = "mean",
                                         bins = kbins)
    Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)
    return (kvals,Abins)


In [ ]:
image = output_image.copy()
kvals, Abins =  power_spectrum(image)

pl.loglog(kvals, Abins)
pl.xlabel("$k$")
pl.ylabel("$P(k)$")
pl.tight_layout()
plt.show()

In [ ]:
kvals_ls = []
abins_ls = []
for t in trange(5,41,5):
    image = gel[t,20,:,:].astype('uint16')
    output_image = autocorr(image)
    image = output_image.copy()
    kvals, Abins =  power_spectrum(image)
    kvals_ls.append(kvals)
    abins_ls.append(Abins)

    pl.loglog(kvals, Abins, label=str(t))
    pl.xlabel("$k$")
    pl.ylabel("$P(k)$")
    pl.tight_layout()
    pl.loglog(kvals, Abins, ls="None")

df = pd.DataFrame(columns=['kval','abins'])
df['kval'] = kvals_ls
df['abins'] = abins_ls
df.to_csv(GRAPH_PATH + 'spectrum/spectrum.csv')
plt.title('Power Spectrum')
plt.legend()
plt.show()

In [ ]:
#plt.imshow(image, cmap='Set1')
plt.plot(np.arange(10), np.arange(10)/10, color = cm.get_cmap('Set1'))
plt.show()

In [ ]:
zy = gel[5][:,:,256]
plt.imshow(zy)
plt.colorbar()
plt.show()


In [ ]:
plt.hist(zy[:,40])

In [ ]:
plt.plot(zy[:,10])
plt.show()

In [ ]:
from skimage import filters
filters.threshold_li(zy[25:,10])

In [ ]:
zy = gel[20][:,:,444]
plt.imshow(zy)
plt.colorbar()
plt.show()


In [ ]:
plt.hist(zy[:,40])


In [ ]:
plt.plot(zy[:,40])
plt.show()

In [ ]:
h = np.zeros((len(gel), gel.shape[2], gel.shape[3]))


def p(time):
    tp = movie_structure.TimePoint(gel[time])
    h[time] = tp.set_height()

with ThreadPoolExecutor() as e:
    for t in trange(len(gel)):
        e.submit(p,t)



In [ ]:
np.save(MOVIE_PATH + 'tmp/h.npy',h)

In [ ]:
mean_height = np.zeros(len(h))
for t in trange(len(h)):
    mean_height[t] = np.nanmean(h[t])

In [ ]:
plt.scatter(range(len(h)),mean_height)
plt.xlabel('Time')
plt.ylabel('Height')
plt.show()

In [ ]:
g = np.load(MOVIE_PATH +'tmp/gel_uint32.npy')

In [ ]:
g.max()

In [ ]:
viewer.add_image(g[0,:,:,20], name='uint32')

In [ ]:
importlib.reload(preprocessing)

In [ ]:
gel = preprocessing.normalize_to_background(gel,0,20,(0,0),(10,10))

In [ ]:
np.save(MOVIE_PATH +'tmp/gel_norm.npy' , gel)

In [ ]:
gel[:,0:20,:,:] = gel[:,-20:,:,:].copy()

In [ ]:
importlib.reload(movie_structure)


In [ ]:
df = pd.read_csv(MOVIE_PATH + 'height.csv')

data = df.drop(['order','Unnamed: 0'],axis=1)
ax = sns.heatmap(data,xticklabels =50, yticklabels=20, cmap ='plasma')
plt.xlabel('X')
plt.ylabel('Time')
plt.title('Gel Height')
ax.invert_yaxis()


$\bar{h}$


In [ ]:
df = pd.read_csv(MOVIE_PATH + 'height.csv')



data = df.drop(['order','Unnamed: 0'],axis=1)
data['time'] = pd.read_csv(MOVIE_PATH +'np/xtime.csv')['time']
data['combined'] =data.apply(lambda row: [(row['time'], e) for e in row.drop('time')], axis = 1)
# take all list in data['combined'] and make it one list
values = data['combined'].apply(pd.Series).stack().reset_index(drop=True)

graph_df = pd.DataFrame(columns=['x','y'])
graph_df['Time'] = values.apply(lambda x:x[0])
graph_df['Mean Height'] = values.apply(lambda x:x[1])
sns.lineplot(data=graph_df, x = 'Time', y = 'Mean Height', estimator='mean' ,errorbar = 'sd')
plt.xlabel('Time')
plt.ylabel('h')
plt.title(r'$\bar{h}$')

In [ ]:

df = pd.read_csv(MOVIE_PATH + 'height.csv')

data = df.drop(['order','Unnamed: 0'],axis=1).to_numpy()
for row in data:
    row-= np.nanmean(row)
data = pd.DataFrame(data)
data['time'] = pd.read_csv(MOVIE_PATH +'np/xtime.csv')['time']
data['combined'] =data.apply(lambda row: [(row['time'], e) for e in row.drop('time')], axis = 1)
# take all list in data['combined'] and make it one list
values = data['combined'].apply(pd.Series).stack().reset_index(drop=True)

graph_df = pd.DataFrame(columns=['x','y'])
graph_df['Time'] = values.apply(lambda x:x[0])
graph_df['Delta Height'] = values.apply(lambda x:x[1])
sns.lineplot(data=graph_df, x = 'Time', y = 'Delta Height', estimator='mean' ,errorbar = 'sd')
plt.title(r'$h - \bar{h}$')
plt.ylabel(r'$h - \bar{h}$')
plt.xlabel('Time')






In [ ]:
data['mean']

In [ ]:
import seaborn as sns
df = pd.read_csv(MOVIE_PATH + 'square.csv')

data = df.drop(['order','Unnamed: 0'],axis=1)
data = data.applymap(lambda x: x/512)
ax = sns.heatmap(data,xticklabels =50, yticklabels=20, cmap ='plasma')
plt.xlabel('X')
plt.ylabel('Time')
plt.title('Gel Square deviation from mean height')
ax.invert_yaxis()



In [ ]:
df = pd.read_csv(MOVIE_PATH + 'curvature.csv')

data = df.drop(['order','Unnamed: 0'],axis=1)
ax = sns.heatmap(data,xticklabels =50, yticklabels=20, cmap ='plasma')
plt.xlabel('X')
plt.ylabel('Time')
plt.title('Abs curvature of boundary')
ax.invert_yaxis()


In [ ]:
df = pd.read_csv(MOVIE_PATH + 'nan.csv')
data = df.drop(['order','Unnamed: 0'],axis=1)
data['time'] = pd.read_csv(MOVIE_PATH +'np/xtime.csv')['time']

sns.lineplot(data=data, x='time', y='0')



In [ ]:
data = df.drop(['order'],axis=1)
sns.heatmap(data)

In [ ]:
importlib.reload(seaborne)


In [ ]:
fig, axes = plt.subplots(2,5, figsize=(30,12))

t=10
for row in axes:
    for col in row:
        if t >90: continue
        tp = movie_structure.TimePoint(gel[t], mask[t])
        surface = tp.set_height_surface()
        sns.heatmap(surface - np.mean(surface),ax= col, cmap= 'seismic', vmin = -200, vmax =200, xticklabels=50, yticklabels=50, square=True)
        col.set_title('Surface height deviation Time point %d'%t, fontsize = 16)
        t+=10

plt.show()


$curvature = \frac{\frac{\partial^2 z}{\partial x^2} \frac{\partial^2 z}{\partial y^2} - \left(\frac{\partial^2 z}{\partial x \partial y}\right)^2}{\left(1 + \left(\frac{\partial z}{\partial x}\right)^2 + \left(\frac{\partial z}{\partial y}\right)^2\right)^{3/2}}$


In [ ]:
df = pd.read_csv(MOVIE_PATH + 'height.csv')
data = df.drop(['order','Unnamed: 0'],axis=1).to_numpy()
#save data
np.save(MOVIE_PATH + 'np/height.npy', data)


In [ ]:
#load data

data = np.load(MOVIE_PATH + 'np/height.npy')

In [ ]:
import numpy as np
from scipy import interpolate
#from scipy.interpolate import RegularGridInterpolator
def replace_nan_with_interpoation(z, sigma = 5):
    img = z.copy()
    mask = np.isnan(img)
    img[mask] = np.nanmean(img)
    g = gaussian(img, sigma)
    img[mask] = g[mask]


    return img



In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import numpy as np
#import ndimage from scipy
from scipy import ndimage
from skimage.filters import threshold_otsu, threshold_local, threshold_mean, threshold_minimum, threshold_triangle, threshold_yen, threshold_li
step = 5
x = np.arange(0,512, step)
y = np.arange(0,512, step)
X, Y = np.meshgrid(x, y)
plots_range = [20] #range(5, 95,10)
#fig = make_subplots(rows=5, cols=2, specs=[[{"type": "surface"},{"type": "surface"}]]*5, subplot_titles=['Surface plot %d'%t for t in plots_range])

fig = make_subplots(rows=1, cols=1, specs=[[{"type": "surface"}]], subplot_titles=['Surface plot %d'%t for t in plots_range])

fig.update_layout(height=2000, width=800)
subplot_row = 1
subplot_col = 1

for t in plots_range:


    max_z = 100
    row = data[t].copy()
    #show histogram of row
    #plt.hist(row.flatten(), bins=100)
    #plt.title('Histogram of row %d'%t)
    #plt.show()

    Z = row.reshape(512,512)
    Z = Z[::step,::step]


    #replace nan with mean
    Z[np.isnan(Z)] = np.nan
    Z = replace_nan_with_interpoation(Z)

    thresh = threshold_yen(Z)
    print('first threshold of time %d is %d'%(t,thresh))
    if (Z<thresh).sum() > (Z>thresh).sum():
        print('surface below thresh  %d'%t)
        Z[Z > thresh] = np.nan
        Z = replace_nan_with_interpoation(Z)
        #thresh = threshold_otsu(Z)
        #Z[Z < thresh] = np.nan
    else:
        #surface above thresh
        mask_nan = Z < thresh
        Z[mask_nan] = np.nan
        #Z = replace_nan_with_interpoation(Z)
        thresh = threshold_yen(Z[~np.isnan(Z)])
        print('second threshold of time %d is %d'%(t,thresh))
        Z[Z> thresh] = np.nan
        Z = replace_nan_with_interpoation(Z)
        Z[mask_nan] = np.nan



    Z = gaussian(Z, 5)


    #make surface subplots with 10 rows and 1 column
    #make spec for each row

    trace = go.Surface(x=X, y=Y, z=Z, cmin=0, cmax=max_z )
    fig.append_trace(trace, row=subplot_row, col=subplot_col)


    #enlarge subplot size

    #add title to each subplot
    if subplot_col==2:
        subplot_row +=1
        subplot_col = 1
    else:
        subplot_col +=1



fig.update_layout(
    title_text="Surface plot %d"%t, scene=dict(zaxis=dict(range=[0, max_z])),
    margin=dict(l=5, r=5, t=5, b=5))

fig.show()



In [ ]:
import numpy as np
#import ndimage from scipy
from scipy import ndimage
from skimage.filters import threshold_otsu, threshold_local, threshold_mean, threshold_minimum, threshold_triangle, threshold_yen, threshold_li
from mpl_toolkits import mplot3d

import matplotlib.pyplot as plt
subplot_row = 1
subplot_col = 1

step =1
x = np.arange(0,512, step)
y = np.arange(0,512, step)
X, Y = np.meshgrid(x, y)
plots_range = range(75, 85,1)
#fig = make_subplots(rows=5, cols=2, specs=[[{"type": "surface"},{"type": "surface"}]]*5, subplot_titles=['Surface plot %d'%t for t in plots_range])

#make subplots 5X2
fig, ax = plt.subplots(5,2, subplot_kw={"projection": "3d"}, figsize=(30,30))

for t in plots_range:


    max_z = 100
    row = data[t].copy()
    #show histogram of row
    #plt.hist(row.flatten(), bins=100)
    #plt.title('Histogram of row %d'%t)
    #plt.show()

    Z = row.reshape(512,512)
    Z = Z[::step,::step]


    #replace nan with mean
    Z[np.isnan(Z)] = np.nan
    Z = replace_nan_with_interpoation(Z)

    thresh = threshold_yen(Z)
    print('first threshold of time %d is %d'%(t,thresh))
    if (Z<thresh).sum() > (Z>thresh).sum():
        print('surface below thresh  %d'%t)
        Z[Z > thresh] = np.nan
        Z = replace_nan_with_interpoation(Z)
        #thresh = threshold_otsu(Z)
        #Z[Z < thresh] = np.nan
    else:
        #surface above thresh
        mask_nan = Z < thresh
        Z[mask_nan] = np.nan
        #Z = replace_nan_with_interpoation(Z)
        thresh = threshold_yen(Z[~np.isnan(Z)])
        print('second threshold of time %d is %d'%(t,thresh))
        Z[Z> thresh] = np.nan
        Z = replace_nan_with_interpoation(Z)
        Z[mask_nan] = np.nan



    Z = gaussian(Z, 5)


    ax[subplot_row-1, subplot_col-1].plot_surface(X, Y, Z,cmap='viridis', edgecolor='none')
    ax[subplot_row-1, subplot_col-1].set_title('Surface plot %d'%t)


    #make surface subplots with 10 rows and 1 column
    #make spec for each row


    #enlarge subplot size

    #add title to each subplot
    if subplot_col==2:
        subplot_row +=1
        subplot_col = 1
    else:
        subplot_col +=1

plt.show()




In [ ]:

from skimage.filters import try_all_threshold

img = Z.copy()
#replace all nan with 0
img[np.isnan(img)] = 0


fig, ax = try_all_threshold(img, figsize=(10, 8), verbose=False)
plt.show()

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
from scipy import ndimage, datasets

import matplotlib.pyplot as plt

fig = plt.figure()

#plt.gray()  # show the filtered result in grayscale

ax1 = fig.add_subplot(121)  # left side

ax2 = fig.add_subplot(122)  # right side
ax2 = fig.add_subplot(123)  # right side


ascent = Z

result = ndimage.gaussian_gradient_magnitude(ascent, sigma=1)

ax1.imshow(ascent)

ax2.imshow(result)
#ax3.imshow(g)
plt.show()